In [1]:
import sys
import pandas as pd


# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

from connect_platform import connect_telemed

import numpy as np
from log import log

In [2]:
import datetime

In [3]:
sql = """
select 	patient.id patient_id,
		a.id app_id,
		date(a.finished at time zone 'UTC') finished
from patient 
inner join appointment a on a.patient_id = patient.id
where 
	  a.good
      and not (
      	a.patient_id in (
	      	select patient_id
	      	from patient_categories pcat
	      	inner join reference rf
	      	   on rf.id=pcat.reference_id
	      	where rf.code='TEST'
	      	)
	    or lower(a.report_comment)='тест'
      	)
"""

In [4]:
conn = connect_telemed()
df: pd.DataFrame = conn.query(sql)
conn.close()

2020-05-22 18:42:05 connect to postgres database using config file "/home/sergey/drclinics/common/../.credentials/telemed/prom.cfg"
2020-05-22 18:42:05 creating ssh tunnel to 172.16.100.19 as root...
2020-05-22 18:42:06 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 33951
2020-05-22 18:42:06 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-05-22 18:42:06 @telemed query:
                    select 	patient.id patient_id,
				    		a.id app_id,
				    		date(a.finished at time zone 'UTC') finished
				    from patient 
				    inner join appointment a on a.patient_id = patient.id
				    where 
				    	  a.good
				          and not (
				          	a.patient_id in (
				    	      	select patient_id
				    	      	from patient_categories pcat
				    	      	inner join reference rf
				    	      	   on rf.id=pcat.reference_id
				    	  

In [5]:
df.head()

,patient_id,app_id,finished
0,1190533,66663,2019-08-29
1,1648936,84449,2019-11-27
2,1192092,76411,2019-10-21
3,1170981,66666,2019-08-29
4,2919271,119907,2020-03-19


In [6]:
df_cons_total = df.groupby(['patient_id'])[['app_id']].count().reset_index().rename(columns={'app_id':'cons_total'})

In [7]:
#df_cons_total.head()

In [8]:
# patients with only 1 consultation
len(df_cons_total[df_cons_total['cons_total'] == 1])

40250

In [9]:
# total patients
len(df.patient_id.unique())
#len(df_cons)

68708

In [10]:
# total consultations
len(df.app_id.unique())

162748

In [11]:
"""
d = {
    'total consultations': [len(df.app_id.unique()), 1],
    'total patients': [len(df.patient_id.unique()), 2],
    'patients with 1 consultation':[len(df_cons_total[df_cons_total['cons_total'] == 1]), 3]
    
}
"""

"\nd = {\n    'total consultations': [len(df.app_id.unique()), 1],\n    'total patients': [len(df.patient_id.unique()), 2],\n    'patients with 1 consultation':[len(df_cons_total[df_cons_total['cons_total'] == 1]), 3]\n    \n}\n"

In [12]:
#report3 = pd.DataFrame.from_dict(d, orient='index').rename(columns={0:'Total', 1:'WEB start'})

In [13]:
#report3

In [14]:
df_cons_web = df[df['finished'] >= datetime.datetime.strptime('2020-03-30', '%Y-%m-%d').date()]

In [16]:
df_cons_web_group = df_cons_web.groupby(['patient_id'])[['app_id']].count().reset_index().rename(columns={'app_id':'cons_total'})

In [17]:
#df_cons_web_group.head()

In [18]:
# WEB patients with only 1 consultation
len(df_cons_web_group[df_cons_web_group['cons_total'] == 1])

23398

In [19]:
# WEB total patients
len(df_cons_web.patient_id.unique())
#len(df_cons)

37763

In [20]:
# WEB total consultations
len(df_cons_web.app_id.unique())

78558

In [21]:
total_con = len(df.app_id.unique())
total_con_web = len(df_cons_web.app_id.unique())

total_pat = len(df.patient_id.unique())
total_pat_web = len(df_cons_web.patient_id.unique())

pat_solo = len(df_cons_total[df_cons_total['cons_total'] == 1])
pat_solo_web = len(df_cons_web_group[df_cons_web_group['cons_total'] == 1])

d = {
    'Всего консультаций': [total_con, 
                           total_con_web,
                           '-',
                           '-'],
    'Всего пациентов': [total_pat, 
                        total_pat_web,
                        '-',
                        '-'],
    'Пациентов с 1-ой консультацией': [pat_solo, 
                                       pat_solo_web,
                                       round(pat_solo * 100 / total_pat, 2),
                                       round(pat_solo_web * 100 / total_pat_web, 2)]
    
}

In [22]:
report3 = pd.DataFrame.from_dict(d, orient='index').rename(columns={0:'Всего', 
                                                                    1:'Из них с начала WEB',
                                                                    2:'Пациентов с 1-ой консультацией, %',
                                                                    3:'Пациентов с 1-ой косультацией с начала WEB %'})

In [23]:
report3

,Всего,Из них с начала WEB,"Пациентов с 1-ой консультацией, %",Пациентов с 1-ой косультацией с начала WEB %
Всего консультаций,162748,78558,-,-
Всего пациентов,68708,37763,-,-
Пациентов с 1-ой консультацией,40250,23398,58.58,61.96


In [24]:
global writer
writer = None

In [25]:
def corr_width(x: int):
    if x <= 7:
        return 7
    elif x >=30:
        return 30
    return x

def auto_columns_width(file_name, data, sheet_name, formatting=[]):
    
    # writer
    global writer
    if not writer:
        writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    
    # drop pandas RangeIndex
    if isinstance(data.index, pd.core.indexes.range.RangeIndex):
        data.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        data.to_excel(writer, sheet_name=sheet_name)
    
    # select current worksheet
    worksheet = writer.sheets[sheet_name]
    
    # read columns
    cols = list(data.columns)
    cols_iter = list(data.columns)
    
    # datacolumns to str
    for i in range(len(cols)):
        if isinstance(cols[i], datetime.date):
            cols[i] = cols[i].strftime('%Y-%m-%d')
    
    # if index exists = set k=1
    k = 0
    if data.index.dtype == object:
        k = 1
    
    # setting index length
    worksheet.set_column(0, 0, max(data.index.astype(str).map(len)) * 1.2)
    
    # setting columns length
    for i, col in enumerate(cols, k):
        
        if data[cols_iter[i-k]].dtype == int:
            col_len = corr_width(len(col)) * 1.2
        else:
            # эта помойка выбирает что длиннее - название столбца или какое-то значение внутри него
            col_len = max(corr_width(max(data[(cols_iter[i-k])].fillna(0).astype(str).map(len))), len(col))
            col_len *= 1.2
        
        print(i, col, col_len)
        worksheet.set_column(i, i, col_len)

In [26]:
auto_columns_width('only-one-con.xlsx', report3, 'data')

writer.save()

1 Всего 8.4
2 Из них с начала WEB 22.8
3 Пациентов с 1-ой консультацией, % 39.6
4 Пациентов с 1-ой косультацией с начала WEB % 52.8
